<a href="https://colab.research.google.com/github/gstdl/Logistics_Analysis/blob/master/Logistics%20Analysis%20using%20%20Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab.files import upload
upload = upload() # Upload Kaggle API Key

Saving kaggle.json to kaggle.json


In [ ]:
!pip install kaggle # Installs Kaggle
!mkdir kaggle # Makes new directory 'kaggle'
!mkdir ../root/.kaggle # Makes new directory '../root/.kaggle'
!cp kaggle.json ~/.kaggle/kaggle.json # Copies kaggle API key to the new directory
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c open-shopee-code-league-logistic # Downloads the dataset as a zip file
 # Move current working directory
%cd /content/{/content}/competitions/open-shopee-code-league-logistic

- path is now set to: {/content}
  0% 0.00/9.01k [00:00<?, ?B/s]
100% 9.01k/9.01k [00:00<00:00, 15.5MB/s]
 96% 367M/381M [00:03<00:00, 163MB/s]
100% 381M/381M [00:03<00:00, 107MB/s]
/content/{/content}/competitions/open-shopee-code-league-logistic


In [ ]:
!ls

delivery_orders_march.csv.zip  SLA_matrix.xlsx


In [ ]:
!unzip delivery_orders_march.csv.zip

Archive:  delivery_orders_march.csv.zip
  inflating: delivery_orders_march.csv  


In [ ]:
!ls

delivery_orders_march.csv  delivery_orders_march.csv.zip  SLA_matrix.xlsx


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from tqdm import tqdm

tqdm.pandas()

In [ ]:
df = pd.read_csv('delivery_orders_march.csv')
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [ ]:
min(df['pick'])

1583079143

In [ ]:
pd.to_datetime(1583079143, unit= 's')

Timestamp('2020-03-01 16:12:23')

In [ ]:
for col in ['pick', '1st_deliver_attempt', '2nd_deliver_attempt']:
    df[col] = (pd.to_datetime(df[col], unit= 's') + pd.offsets.DateOffset(hours= 8)).apply(lambda x: x.date())

df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04,2020-03-06,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05,2020-03-07,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04,2020-03-06,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [ ]:
df[['pick', '1st_deliver_attempt', '2nd_deliver_attempt']].agg(['min', 'max'])

,pick,1st_deliver_attempt
min,2020-03-02,2020-03-02
max,2020-04-06,2020-04-07


In [ ]:
df['2nd_deliver_attempt'].dropna().agg(['min', 'max'])

min    2020-03-04
max    2020-04-08
Name: 2nd_deliver_attempt, dtype: object

In [ ]:
holidays = [
    '2020-03-08', #(Sunday);
    '2020-03-25', #(Wednesday);
    '2020-03-30', #(Monday);
    '2020-03-31'  #(Tuesday)
]

holidays = [pd.to_datetime(i) for i in holidays]

In [ ]:
for i in range(1,5):
    holidays.append((holidays[0] + pd.offsets.DateOffset(days= 7*i)))
holidays.sort()
holidays

[Timestamp('2020-03-08 00:00:00'),
 Timestamp('2020-03-15 00:00:00'),
 Timestamp('2020-03-22 00:00:00'),
 Timestamp('2020-03-25 00:00:00'),
 Timestamp('2020-03-29 00:00:00'),
 Timestamp('2020-03-30 00:00:00'),
 Timestamp('2020-03-31 00:00:00'),
 Timestamp('2020-04-05 00:00:00')]

In [ ]:
# df_matrix =  [i.split('\t') for i in '''Metro Manila 	Luzon 	Visayas 	Mindanao
# Metro Manila 	3 	5 	7 	7
# Luzon 	5 	5 	7 	7
# Visayas 	7 	7 	7 	7
# Mindanao 	7 	7 	7 	7'''.split('\n')]
# cities = df_matrix[0]
# matrix = np.empty([4,4])
# for row in range(1, 5):
#     for col in range(1, 5):
#         matrix[row-1, col-1] = df_matrix[row][col]
df_matrix = pd.read_excel('SLA_matrix.xlsx').to_numpy()
cities = df_matrix[0, 2:]
matrix = df_matrix[1:5, 2:]
df_matrix = pd.DataFrame(matrix, cities, cities).progress_apply(lambda x: [int(i.replace(' working days', '')) for i in x])
df_matrix

100%|██████████| 4/4 [00:00<00:00, 914.59it/s]


,Metro Manila,Luzon,Visayas,Mindanao
Metro Manila,3,5,7,7
Luzon,5,5,7,7
Visayas,7,7,7,7
Mindanao,7,7,7,7


In [ ]:
import re

city_findall = re.compile('|'.join([c.strip() for c in cities]).lower())

def find_city(col):
    c = city_findall.findall(col.lower())
    return c[-1].title()

def expected_delivery(cols):
    try:
        return df_matrix.loc[cols['seller_city'], cols['buyer_city']]
    except:
        return np.nan

In [ ]:
df['buyer_city'] = df['buyeraddress'].progress_apply(find_city)
df['seller_city'] = df['selleraddress'].progress_apply(find_city)
df['expected_delivery_time_days'] = df.progress_apply(expected_delivery, axis= 1)

df.head()

100%|██████████| 3176313/3176313 [02:05<00:00, 25351.51it/s]


,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,buyer_city,seller_city,expected_delivery_time_days
0,2215676524,2020-03-02,2020-03-05,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,Metro Manila,Metro Manila,3
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",Metro Manila,Metro Manila,3
2,2220979489,2020-03-04,2020-03-06,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",Metro Manila,Metro Manila,3
3,2221066352,2020-03-05,2020-03-07,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",Metro Manila,Metro Manila,3
4,2222478803,2020-03-04,2020-03-06,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,Luzon,Metro Manila,5


In [ ]:
def delivery_time(pick, deliver):
    time = (deliver - pick).days
    ## alternative 1
    dates = [pick + pd.offsets.DateOffset(days= 1)]
    while dates[-1] < deliver:
        dates.append(dates[-1] + pd.offsets.DateOffset(days= 1))
    time -= len(np.intersect1d(dates, holidays))
    ## alternative 2
    # offsets = [1 if pick + pd.offsets.DateOffset(days= i + 1) in holidays else 0 for i in range(time - 1)]
    # time -= sum(offsets)
    ## alternative 3
    #offset = 0
    #date = pick
    #while date < deliver:
    #  date += pd.offsets.DateOffset(days= 1)
    #  if date in holidays:
    #      offset += 1
    #time -= offset
    return time
    
def deliver_1st(cols):
    return delivery_time(cols['pick'], cols['1st_deliver_attempt'])
 
def deliver_2nd(cols):
    if pd.isna(cols['2nd_deliver_attempt']):
        return np.nan
    return delivery_time(cols['1st_deliver_attempt'], cols['2nd_deliver_attempt'])

In [ ]:
df['1st_delivery_time'] = df.progress_apply(deliver_1st, axis= 1)
df['2nd_delivery_time'] = df.progress_apply(deliver_2nd, axis= 1)
df.head()

100%|██████████| 3176313/3176313 [06:34<00:00, 8043.33it/s]


,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,buyer_city,seller_city,expected_delivery_time_days,1st_delivery_time,2nd_delivery_time
0,2215676524,2020-03-02,2020-03-05,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,Metro Manila,Metro Manila,3,3,NaN
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",Metro Manila,Metro Manila,3,2,3.0
2,2220979489,2020-03-04,2020-03-06,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",Metro Manila,Metro Manila,3,2,NaN
3,2221066352,2020-03-05,2020-03-07,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",Metro Manila,Metro Manila,3,2,NaN
4,2222478803,2020-03-04,2020-03-06,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,Luzon,Metro Manila,5,2,NaN


In [ ]:
df.isna().sum()

orderid                              0
pick                                 0
1st_deliver_attempt                  0
2nd_deliver_attempt            1819311
buyeraddress                         0
selleraddress                        0
buyer_city                           0
seller_city                          0
expected_delivery_time_days          0
1st_delivery_time                    0
2nd_delivery_time              1819311
dtype: int64

In [ ]:
df['buyer_city'].value_counts()

Metro Manila    1560698
Luzon            804888
Visayas          479595
Mindanao         331132
Name: buyer_city, dtype: int64

In [ ]:
df['seller_city'].value_counts()

Metro Manila    3176312
Luzon                 1
Name: seller_city, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3176313 entries, 0 to 3176312
Data columns (total 11 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   orderid                      int64  
 1   pick                         object 
 2   1st_deliver_attempt          object 
 3   2nd_deliver_attempt          object 
 4   buyeraddress                 object 
 5   selleraddress                object 
 6   buyer_city                   object 
 7   seller_city                  object 
 8   expected_delivery_time_days  int64  
 9   1st_delivery_time            int64  
 10  2nd_delivery_time            float64
dtypes: float64(1), int64(3), object(7)
memory usage: 266.6+ MB


In [ ]:
def is_late(cols):
    if cols['1st_delivery_time'] > cols['expected_delivery_time_days']:
        return 1
    elif pd.notna(cols['2nd_delivery_time']) and cols['2nd_delivery_time'] > 3:
        return 1
    else:
        return 0

In [ ]:
df['is_late'] = df.progress_apply(is_late, axis= 1)
 
df.head()

100%|██████████| 3176313/3176313 [02:12<00:00, 24061.65it/s]


,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,buyer_city,seller_city,expected_delivery_time_days,1st_delivery_time,2nd_delivery_time,is_late
0,2215676524,2020-03-02,2020-03-05,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,Metro Manila,Metro Manila,3,3,NaN,0
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",Metro Manila,Metro Manila,3,2,3.0,0
2,2220979489,2020-03-04,2020-03-06,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",Metro Manila,Metro Manila,3,2,NaN,0
3,2221066352,2020-03-05,2020-03-07,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",Metro Manila,Metro Manila,3,2,NaN,0
4,2222478803,2020-03-04,2020-03-06,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,Luzon,Metro Manila,5,2,NaN,0


In [ ]:
df_sub = df.copy()[['orderid', 'is_late']]
df_sub.head()

,orderid,is_late
0,2215676524,0
1,2219624609,0
2,2220979489,0
3,2221066352,0
4,2222478803,0


In [ ]:
df_sub.to_csv('submission.csv', index= False)

In [ ]:
!kaggle competitions submit -c open-shopee-code-league-logistic -f submission.csv -m "sub 02:45"

100% 41.2M/41.2M [00:00<00:00, 56.4MB/s]
Successfully submitted to [Open] Shopee Code League - Logistics